# Food Satefy Letter Grade Clustering & Prediction
This project is going to use public data for health inspections in NYC restaurants to determine what the predicted letter grade of a restaurant would be based on different demographics such as cuisine, location, etc.


In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
import math
from sklearn.feature_extraction.text import TfidfVectorizer #to create 1/0s from string fields
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Import Data Sets
NYC_Inspection_Results = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results - FULL DATASET.csv')
NYC_Inspection_Results.head()
NYC_Inspection_Results.info()
NYC_Inspection_Results.dtypes

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,50100093,YIN JI CHANG FEN,Manhattan,91,BAYARD STREET,10013.0,2122274888,Chinese,11/6/2019,Violations were cited in the following area(s).,...,12/13/2020,Pre-permit (Operational) / Initial Inspection,40.715862,-73.999199,103.0,1.0,2900.0,1077449.0,1.001640e+09,MN27
1,40879245,GRAND HYATT NEW YORK HOTEL,Manhattan,0,@ GRAND CENTRAL,NaN,6462136774,American,5/24/2018,Violations were cited in the following area(s).,...,12/13/2020,Cycle Inspection / Initial Inspection,0.000000,0.000000,NaN,NaN,NaN,NaN,1.000000e+00,NaN
2,50047078,POKE FRESH SUSHI,Manhattan,1588,YORK AVENUE,10028.0,2125355798,Chinese/Japanese,7/18/2019,Violations were cited in the following area(s).,...,12/13/2020,Cycle Inspection / Re-inspection,40.774397,-73.948225,108.0,5.0,13600.0,1051209.0,1.015800e+09,MN32
3,41225392,BAYARD'S ALEHOUSE,Manhattan,533,HUDSON STREET,10014.0,2129890313,American,9/25/2019,Violations were cited in the following area(s).,...,12/13/2020,Cycle Inspection / Initial Inspection,40.734329,-74.006210,102.0,3.0,7500.0,1011756.0,1.006310e+09,MN23
4,50013970,CAFFE BENE,Manhattan,39,WEST 32 STREET,10001.0,2015431827,Café/Coffee/Tea,12/30/2016,Violations were cited in the following area(s).,...,12/13/2020,Cycle Inspection / Initial Inspection,40.747646,-73.986650,105.0,4.0,7600.0,1015844.0,1.008340e+09,MN17


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399906 entries, 0 to 399905
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  399906 non-null  int64  
 1   DBA                    399336 non-null  object 
 2   BORO                   399906 non-null  object 
 3   BUILDING               398676 non-null  object 
 4   STREET                 399817 non-null  object 
 5   ZIPCODE                393172 non-null  float64
 6   PHONE                  399886 non-null  object 
 7   CUISINE DESCRIPTION    399906 non-null  object 
 8   INSPECTION DATE        399906 non-null  object 
 9   ACTION                 396869 non-null  object 
 10  VIOLATION CODE         392573 non-null  object 
 11  VIOLATION DESCRIPTION  389206 non-null  object 
 12  CRITICAL FLAG          389206 non-null  object 
 13  SCORE                  381501 non-null  float64
 14  GRADE                  202028 non-nu

CAMIS                      int64
DBA                       object
BORO                      object
BUILDING                  object
STREET                    object
ZIPCODE                  float64
PHONE                     object
CUISINE DESCRIPTION       object
INSPECTION DATE           object
ACTION                    object
VIOLATION CODE            object
VIOLATION DESCRIPTION     object
CRITICAL FLAG             object
SCORE                    float64
GRADE                     object
GRADE DATE                object
RECORD DATE               object
INSPECTION TYPE           object
Latitude                 float64
Longitude                float64
Community Board          float64
Council District         float64
Census Tract             float64
BIN                      float64
BBL                      float64
NTA                       object
dtype: object

In [3]:
#New fields to be added
NYC_Inspection_Results['INSPECTION DATE'] = pd.to_datetime(NYC_Inspection_Results['INSPECTION DATE'])
NYC_Inspection_Results['INSPECTION YEAR'] = pd.DatetimeIndex(NYC_Inspection_Results['INSPECTION DATE']).year
NYC_Inspection_Results['INSPECTION TYPE'].value_counts()

Cycle Inspection / Initial Inspection                          233063
Cycle Inspection / Re-inspection                                95434
Pre-permit (Operational) / Initial Inspection                   28303
Pre-permit (Operational) / Re-inspection                        12223
Administrative Miscellaneous / Initial Inspection                7384
Cycle Inspection / Reopening Inspection                          4389
Pre-permit (Non-operational) / Initial Inspection                3497
Smoke-Free Air Act / Initial Inspection                          1977
Administrative Miscellaneous / Re-inspection                     1969
Trans Fat / Initial Inspection                                   1519
Pre-permit (Operational) / Compliance Inspection                 1267
Calorie Posting / Initial Inspection                             1219
Inter-Agency Task Force / Initial Inspection                     1076
Pre-permit (Operational) / Reopening Inspection                  1063
Cycle Inspection / C

In [22]:
NYC_Inspection_Results['INSPECTION TYPE'] = NYC_Inspection_Results['INSPECTION TYPE'].astype(str)

<bound method DataFrame.info of            CAMIS                                DBA       BORO BUILDING  \
0       50100093                   YIN JI CHANG FEN  Manhattan       91   
1       40879245         GRAND HYATT NEW YORK HOTEL  Manhattan        0   
2       50047078                   POKE FRESH SUSHI  Manhattan     1588   
3       41225392                  BAYARD'S ALEHOUSE  Manhattan      533   
4       50013970                         CAFFE BENE  Manhattan       39   
...          ...                                ...        ...      ...   
399901  41672903                 MALONEY & PORCELLI  Manhattan       37   
399902  50059824              TAIKI AND RICE & MISO   Brooklyn      134   
399903  50069734  PRINCE KABAB & CHINESE RESTAURANT     Queens     3756   
399904  41569582                          CHAVELA'S   Brooklyn      736   
399905  40367790                         MCDONALD'S   Brooklyn      943   

                  STREET  ZIPCODE       PHONE CUISINE DESCRIPTION  

In [50]:
#pass the field inspection type to string
NYC_Inspection_Results['INSPECTION TYPE'] = NYC_Inspection_Results['INSPECTION TYPE'].astype(str)

# Instantiate TfidfVectorizer
tv = TfidfVectorizer(max_features=100, stop_words='english',ngram_range = (1,3),min_df=0.02, max_df=0.8)

# Fit the vectorizer and transform the data
tv_transformed = tv.fit_transform(NYC_Inspection_Results['INSPECTION TYPE'])

# Create a DataFrame with these features
tv_df = pd.DataFrame(tv_transformed.toarray(), 
                     columns=tv.get_feature_names()).add_prefix('INS_TYPE_')
# Change fractional amounts to binomials calling the 'ceil' function to round up fractions
tv_df = np.ceil(tv_df) 
print(tv_df.head())

   INS_TYPE_administrative  INS_TYPE_administrative miscellaneous  \
0                      0.0                                    0.0   
1                      0.0                                    0.0   
2                      0.0                                    0.0   
3                      0.0                                    0.0   
4                      0.0                                    0.0   

   INS_TYPE_cycle inspection initial  INS_TYPE_cycle inspection inspection  \
0                                0.0                                   0.0   
1                                1.0                                   0.0   
2                                0.0                                   1.0   
3                                1.0                                   0.0   
4                                1.0                                   0.0   

   INS_TYPE_initial  INS_TYPE_initial inspection  INS_TYPE_inspection initial  \
0               1.0                

In [51]:
#Merge my new fields that were generated using the vectorizer to my main dataframe
df = NYC_Inspection_Results.join(tv_df)
df.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INS_TYPE_operational initial,INS_TYPE_operational initial inspection,INS_TYPE_operational inspection,INS_TYPE_permit,INS_TYPE_permit operational,INS_TYPE_permit operational initial,INS_TYPE_permit operational inspection,INS_TYPE_pre,INS_TYPE_pre permit,INS_TYPE_pre permit operational
0,50100093,YIN JI CHANG FEN,Manhattan,91,BAYARD STREET,10013.0,2122274888,Chinese,2019-11-06,Violations were cited in the following area(s).,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
1,40879245,GRAND HYATT NEW YORK HOTEL,Manhattan,0,@ GRAND CENTRAL,NaN,6462136774,American,2018-05-24,Violations were cited in the following area(s).,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50047078,POKE FRESH SUSHI,Manhattan,1588,YORK AVENUE,10028.0,2125355798,Chinese/Japanese,2019-07-18,Violations were cited in the following area(s).,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,41225392,BAYARD'S ALEHOUSE,Manhattan,533,HUDSON STREET,10014.0,2129890313,American,2019-09-25,Violations were cited in the following area(s).,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50013970,CAFFE BENE,Manhattan,39,WEST 32 STREET,10001.0,2015431827,Café/Coffee/Tea,2016-12-30,Violations were cited in the following area(s).,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
#split my data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'GRADE'], df['GRADE'], test_size=0.10, random_state=42)